# Pre-annotation with Pixano [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixano/pixano/blob/main/notebooks/models/pre_annotation.ipynb)

This notebook allows you to preannotate a whole Pixano dataset using a deep learning model.

You can then browse the inferences and compare them to ground truths in the Pixano Explorer.

## 1. Setting up

### Install dependencies

This notebook requires installing `pixano` as well the models from the complementary [`pixano-inference`](https://github.com/pixano/pixano-inference) module.

If you are running this notebook on your computer, we strongly recommend creating a virtual environment for using Pixano like so:

```shell
conda create -n pixano_env python=3.10
conda activate pixano_env
```

```shell
pip install pixano
pip install pixano-inference@git+https://github.com/pixano/pixano-inference
```

If you are running this notebook in Google Colab, run the cell below to install `pixano` and `pixano-inference`.

In [ ]:
try:
    import google.colab

    ENV = "colab"
    %pip install pixano
    %pip install pixano-inference@git+https://github.com/pixano/pixano-inference
except:
    ENV = "jupyter"

### Load dependencies

In [ ]:
from pathlib import Path

from pixano_inference import pytorch_models, segment_anything, tensorflow_models

from pixano.apps import Explorer

### Download checkpoints
- Segment Anything Model:
    - `default` or `vit_h`: [ViT-H SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth).
    - `vit_l`: [ViT-L SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth).
    - `vit_b`: [ViT-B SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth).
    
## 2. Genererating inferences

With the Pixano Inference module, we provide some useful models from the PyTorch and TensorFlow model hubs, as well as the Segment Anything Model (SAM) from Meta.

You can take a look at how they are implemented to add your own, or reach out to us if you think Pixano Inference could benefit from it, and we will try to add it in a future version. 

### Select a model
- Loading a model:
    - Object Detection (COCO labels):
        - `model = pytorch_models.YOLOv5(size="m")`
        - `model = tensorflow_models.FasterRCNN()`
        - `model = tensorflow_models.EfficientDet()`
    - Instance Segmentation (COCO labels):
        - `model = pytorch_models.MaskRCNNv2()`

    - Semantic Segmentation (VOC labels):
        - `model = pytorch_models.DeepLabV3()`

    - Segment Anything Model (No labels):
        - `model = segment_anything.SAM(checkpoint_path=Path("sam_vit_b_01ec64.pth"), size="b")`

By default, `device` is equal to `"cuda"` for PyTorch models and `"/GPU:0"` for TensorFlow models.

If you would like to run the model on your CPU, please add `device="cpu"` for PyTorch models or `device="/CPU:0"` for TensorFlow models in the arguments in the cell below.

In [ ]:
model = pytorch_models.MaskRCNNv2()

### Select a Pixano format dataset

If you haven't already, please refer to the [dataset notebooks](../datasets/) for information on how to import your dataset to Pixano format.

In [ ]:
library_dir = Path("my_datasets/")
dataset_dir = library_dir / "coco_instances"

views = ["image"]
splits = []

### Generate inferences

In [ ]:
help(model.process_dataset)

In [ ]:
model.process_dataset(
    dataset_dir=dataset_dir,
    process_type="obj",
    views=views,
    splits=splits,
    batch_size=1,
    threshold=0.1,
)

## 3. Browsing the dataset

With the generation complete, you can now browse your dataset with the Pixano Explorer and compare your inferences with ground truths.

You can stop the Explorer app by restarting the notebook.

In [ ]:
explorer = Explorer(library_dir)

In [ ]:
explorer.display()